## Importações

In [ ]:
import pandas as pd
from crewai import Agent, Task, Crew
from langchain_experimental.agents import create_pandas_dataframe_agent
from dotenv import load_dotenv, find_dotenv

from langchain.tools import BaseTool
from crewai.tools import BaseTool
from pydantic import Field
from typing import Any

from langchain_openai import ChatOpenAI
import json
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from fpdf import FPDF
import re
from langchain_core.tools import Tool
import kagglehub
import os
load_dotenv()

## Base de Dados + ETL

In [ ]:
path = kagglehub.dataset_download("felipeesc/shark-attack-dataset")

arquivos = os.listdir(path)
print(f"Arquivos baixados: {arquivos}")

caminho_csv = os.path.join(path, 'attacks.csv')

In [ ]:
# 4. Carrega o DataFrame
df = pd.read_csv(
    caminho_csv, 
    sep=',', 
    on_bad_lines='skip', 
    encoding='latin1'
)

In [ ]:
df.columns

In [ ]:
def preprocess_data(df):
    df.columns = df.columns.str.strip()
    df = df.dropna(subset=['Type', 'Case Number', 'Activity', 'Country'])
    df = df[df.Year < 2018]
    # Seleção de colunas e normalização de nomes
    return df[['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Sex', 'Fatal (Y/N)']]

In [ ]:
df = preprocess_data(df)

## Modelo

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

## Agente 1 - Analista Técnico - Categorizar Atividades

In [ ]:
analista_tecnico = Agent(
    role='Analista de Dados Sênior',
    goal='Classificar dados com precisão cirúrgica',
    backstory='Você foca apenas em resultados estruturados e odeia conversas irrelevantes.', # Isso ajuda a IA a não mandar "Aqui está seu dicionário:"
    llm=llm,
    verbose=True # Mantenha True enquanto testa para ver a IA "pensando"
)

In [ ]:
prompt_classificacao = """
Sua missão é criar um mapeamento de PALAVRAS-CHAVE para categorizar atividades de ataques de tubarão.

REGRAS DE OURO:
1. Analise as atividades e extraia o termo principal (ex: se tiver 'Surfing', 'Surfing a reef break' e 'Paddling', a chave deve ser 'surf').
2. Retorne um JSON simples.
3. Categorias permitidas: 'Esportes de Prancha', 'Natação/Mergulho', 'Pesca', 'Atividades em Barco', 'Interação Provocada'.

DADOS: {top_atividades}
"""

# tarefa_mapeamento = Task(
#     description=prompt_classificacao,
#     agent=analista_tecnico,
#     expected_output="UM OBJETO JSON PURO. Certifique-se de usar aspas DUPLAS para chaves e valores. Não inclua vírgulas após o último item do dicionário."
# )

In [ ]:
contagem = df['Activity'].value_counts(normalize=True) * 100

# 2. Calcular a porcentagem acumulada
acumulado = contagem.cumsum()

# 3. Filtrar as atividades que compõem até 90% (e a primeira que passa de 90%)
atividades = acumulado[acumulado <= 80]

In [ ]:
atividades.head(2)

In [ ]:
top_atividades = atividades.index.tolist()

In [ ]:
tarefa_mapeamento = Task(
    description=f"""
    {prompt_classificacao}
    
    DADOS PARA ANALISAR:
    Aqui estão as atividades mais frequentes: {top_atividades}
    """,
    agent=analista_tecnico,
    expected_output="Um dicionário JSON puro (sem explicações) onde a CHAVE é a atividade original e o VALOR é a categoria."
)

In [ ]:
equipe = Crew(
    agents=[analista_tecnico],
    tasks=[tarefa_mapeamento],
    verbose=False
)

In [ ]:
resultado = equipe.kickoff()

In [ ]:
texto_bruto = str(resultado.raw)

# 1. Tentativa 01: Busca padrão por JSON
json_match = re.search(r'\{.*\}', texto_bruto, re.DOTALL)

if json_match:
    try:
        conteudo = json_match.group(0)
        mapeamento = json.loads(conteudo)
    except:
        # Tentativa 02: Se o JSON falhar, tenta interpretar como dicionário Python literal
        try:
            mapeamento = ast.literal_eval(conteudo)
        except:
            mapeamento = {}
else:
    # Tentativa 03: Se não houver chaves, vamos tentar extrair "Chave: Valor" via Regex
    # Isso salva o seu código se a IA mandar apenas uma lista de texto
    print("Aviso: JSON não encontrado. Tentando extração via Regex de texto plano...")
    linhas = re.findall(r'"?([\w\s/]+)"?:\s*"?([\w\s/]+)"?', texto_bruto)
    mapeamento = {k.strip(): v.strip() for k, v in linhas}

if not mapeamento:
    print("Erro: A IA não devolveu um formato utilizável. Verifique o log do Agente.")

In [ ]:
# Inverte o dicionário para agrupar palavras por categoria
# Ex: {'surf': 'Esportes', 'surfing': 'Esportes'} -> {'Esportes': ['surf', 'surfing']}
categorias_map = {}
for k, v in mapeamento.items():
    categorias_map.setdefault(v, []).append(re.escape(k))

# Função otimizada
def categorizar_veloz(df):
    df['Activity_Category'] = "Outros/Desconhecido"
    for categoria, palavras in categorias_map.items():
        # Cria um padrão regex: (palavra1|palavra2|palavra3)
        padrao = '|'.join(palavras)
        # Aplica em bloco para todas as linhas de uma vez
        mask = df['Activity'].str.contains(padrao, case=False, na=False)
        df.loc[mask, 'Activity_Category'] = categoria
    return df

df = categorizar_veloz(df)

In [ ]:
round(df.Activity_Category.value_counts(normalize=True)*100,0)

## Agente 2 - Escritor Executivo - Estrategista de dados

In [ ]:
# 1. Definição da Ferramenta Robusta
class FerramentaAnalisePandas(BaseTool):
    name: str = "Calculadora_Dados"
    description: str = "Executa comandos no DataFrame. Pode enviar o comando começando com 'df' ou não."

    def _run(self, query: str) -> str:
        try:
            # Remove espaços e aspas extras
            q = str(query).strip().strip("'`\"")
            
            # Se a IA enviou algo começando com "df[", executamos direto o eval(q)
            # Se não, montamos o eval(f"df.{q}")
            if q.startswith("df"):
                resultado = eval(q)
            else:
                resultado = eval(f"df.{q}")
            
            if hasattr(resultado, 'to_frame'): # Melhora legibilidade de Series
                return resultado.to_frame().to_markdown()
            return str(resultado)
        except Exception as e:
            return f"Erro técnico: {e}. Tente simplificar a query."

ferramenta_df = FerramentaAnalisePandas()

# 2. Agente Especialista
# Usamos um backstory mais focado em 'verificação' para evitar alucinações
escritor_executivo = Agent(
    role='Estrategista de Dados Sênior',
    goal='Analisar ataques de tubarão e gerar relatórios executivos baseados em evidências',
    backstory="""Você é um analista rigoroso. Você nunca inventa números.
    Para cada país ou categoria, você usa a Calculadora_Dados para obter o valor exato.
    Sua especialidade é transformar tabelas de dados em insights estratégicos.""",
    llm=llm, # Certifique-se que o gpt-3.5-turbo está definido antes
    tools=[ferramenta_df],
    verbose=True,
    allow_delegation=False
)

# 3. Tarefa com Instruções Passo a Passo (O "pulo do gato")
tarefa_relatorio = Task(
    description="""
    VOCÊ ESTÁ PROIBIDO DE INVENTAR NÚMEROS.
    Se você não conseguir rodar a ferramenta para um país, escreva 'Dados não encontrados'.
    Siga este fluxo de trabalho para construir o relatório:

    1. CATEGORIAS: Use 'Activity_Category.value_counts()' para ver o volume. 
       Depois, use 'groupby("Activity_Category")["Fatal (Y/N)"].value_counts()' para ver mortes.

    2. PAÍSES: Identifique os Top 3 com 'Country.value_counts().head(3)'.

    3. DETALHAMENTO GEOGRÁFICO:
        3.1. Descubra os Top 3 Países: Use 'df["Country"].value_counts().head(3)'
        3.2. Liste as Top 3 Áreas para o País 1: Use 'df[df["Country"]=="NOME1"]["Area"].value_counts().head(3)'
        3.3. Liste as Top 3 Áreas para o País 2: Use 'df[df["Country"]=="NOME2"]["Area"].value_counts().head(3)'
        3.4. Liste as Top 3 Áreas para o País 3: Use 'df[df["Country"]=="NOME3"]["Area"].value_counts().head(3)'
    4. TENDÊNCIA: Veja o volume por ano com 'Year.value_counts().sort_index()'.

    Apenas após receber os textos da ferramenta, monte as tabelas Markdown.
    Se a ferramenta retornar 'Erro', você deve relatar o erro em vez de inventar números.

    REGRAS DE FORMATAÇÃO:
    - O relatório deve ser em Markdown.
    - Use Tabelas para mostrar Países e suas Áreas (Cidades).
    - Crie uma seção de 'Insights de Segurança' baseada nos dados encontrados.

    """,
    agent=escritor_executivo,
    expected_output="Um relatório formal em Markdown com tabelas detalhadas por país/área e métricas de fatalidade."
)

# 4. Execução da Equipe
equipe = Crew(
    agents=[escritor_executivo],
    tasks=[tarefa_relatorio],
    verbose=True
)

resultado_final = equipe.kickoff()

# 5. Salvar e Gerar PDF (Use a função que criamos anteriormente)
# criar_pdf_final(resultado_final.raw)

In [ ]:
resultado_final

## Tarefa Final - Montar Relatório

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def gerar_visualizacoes_completa(df):
    sns.set_theme(style="whitegrid")
    
    # --- Gráfico 1: Categorias ---
    plt.figure(figsize=(10, 5))
    order = df['Activity_Category'].value_counts().index
    sns.countplot(data=df, y='Activity_Category', order=order, palette='viridis')
    plt.ylabel('')
    plt.title('Incidentes por Categoria')
    plt.tight_layout()
    plt.savefig('grafico_categorias.png')
    plt.close()

    # --- Gráfico 2: Fatalidade ---
    plt.figure(figsize=(15, 5))
    df_plot = df[df['Fatal (Y/N)'].isin(['Y', 'N'])]
    
    # Definindo a paleta personalizada
    cores_customizadas = {'Y': 'red', 'N': 'blue'}
    
    sns.countplot(
        data=df_plot, 
        x='Activity_Category', 
        hue='Fatal (Y/N)', 
        palette=cores_customizadas
    )
    
    plt.xlabel('')
    plt.ylabel('')
    plt.title('Fatalidade por Categoria')
    plt.tight_layout()
    plt.savefig('grafico_fatalidade.png')
    plt.close()

    # --- Gráfico 3: Tendência Temporal ---
    plt.figure(figsize=(12, 5))
    df_temporal = df[(df['Year'] > 1950) & (df['Year'] <= 2017)].copy()
    df_temporal = df_temporal.groupby('Year').size().reset_index(name='Contagem')
    sns.lineplot(data=df_temporal, x='Year', y='Contagem', color='#2c3e50', linewidth=2)
    plt.fill_between(df_temporal['Year'], df_temporal['Contagem'], color='#2c3e50', alpha=0.1)
    plt.title('Evolução Temporal dos Ataques (Pós-1950)', fontsize=14)
    plt.xlabel('Ano')
    plt.ylabel('Total de Incidentes')
    plt.tight_layout()
    plt.savefig('grafico_temporal.png')
    plt.close()

    # --- NOVO Gráfico 4: Top 3 Países (Rosca) ---
    plt.figure(figsize=(6, 6))
    
    # Preparação dos dados: Top 3 vs Resto
    counts = df['Country'].dropna().value_counts()
    top_3 = counts.head(3)
    outros = counts.iloc[3:].sum()
    
    labels = list(top_3.index) + ['Outros']
    valores = list(top_3.values) + [outros]
    
    # Cores seguindo o padrão vibrante dos anteriores
    colors = sns.color_palette('viridis', len(labels))
    
    # Criando o gráfico de pizza
    plt.pie(valores, labels=labels, autopct='%1.1f%%', startangle=140, 
            colors=colors, pctdistance=0.75, explode=[0.05, 0, 0, 0], radius=0.9)
    
    # Transformando em Rosca (Donut)
    centre_circle = plt.Circle((0,0), 0.80, fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)
    
    plt.title('Representatividade: Top 3 Países vs Resto do Mundo', fontsize=14)
    plt.tight_layout()
    plt.savefig('grafico_top_paises_rosca.png')
    plt.close()

    print("Visualizações geradas com sucesso!")

# Execute a geração
gerar_visualizacoes_completa(df)

In [ ]:
# TAREFA 2: Relatório Executivo Visual
tarefa_relatorio_visual = Task(
    description="""
    Com base nos dados extraídos na tarefa anterior, monte o relatório final em Markdown.

    
    IMPORTANTE: Use EXATAMENTE estes nomes de arquivos para as imagens:
    - grafico_temporal.png
    - grafico_categorias.png
    - grafico_fatalidade.png
    - grafico_top_paises_rosca.png

    Antes de cada gáfico, escrever principais insights que podem ser observados nos gráficos.

    ESTRUTURA OBRIGATÓRIA:
    1. Título Principal.
    2. Resumo Executivo.
    3. Análise Temporal + ![Tendência Temporal](grafico_temporal.png).
    4. Análise por Atividade + ![Distribuição por Atividade](grafico_categorias.png).
    5. Análise de Fatalidade + ![Fatalidade](grafico_fatalidade.png).
    6. Análise Top 3 Países: Inclua APENAS o gráfico ![Top 3 Países](grafico_top_paises_rosca.png). Não liste os países em texto aqui.
    7. Detalhamento Geográfico: Tabelas com as 3 áreas mais afetadas para os 3 principais países.
    8. Conclusões Finais.
    """,
    agent=escritor_executivo,
    expected_output="Relatório Markdown com gráficos integrados.",
    context=[tarefa_relatorio],
    output_file="relatorio_final.md"
)

In [ ]:
equipe = Crew(
    agents=[escritor_executivo],
    tasks=[tarefa_relatorio, tarefa_relatorio_visual], # Sequência: Dados -> Markdown
    verbose=True
)

resultado = equipe.kickoff()

# Opcional: Se você NÃO usou 'output_file' na Task, mantenha este bloco.
# Se usou 'output_file', este bloco abaixo é apenas uma garantia extra.
with open("Relatorio_Final_Visual.md", "w", encoding="utf-8") as f:
    f.write(str(resultado)) # Use str() ou .raw para garantir o formato texto

In [ ]:
import re

class PDFReport(FPDF): 
    def header(self):
        # Título apenas na primeira página
        if self.page_no() == 1:
            self.set_font('helvetica', 'B', 16)
            self.set_text_color(43, 123, 186)
            # w=0 garante que use a linha toda sem erro de espaço
            self.cell(0, 10, 'RELATÓRIO EXECUTIVO: INCIDENTES COM TUBARÕES', ln=1, align='C')
            self.set_draw_color(43, 123, 186)
            self.line(10, 22, 200, 22)
            self.ln(10)

def criar_pdf_final(texto_markdown):
    pdf = PDFReport()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    
    # 1. LIMPEZA INICIAL
    # Remove aspas triplas de bloco de código e negritos
    texto_limpo = texto_markdown.replace('```markdown', '').replace('```', '').replace('**', '')
    texto_limpo = texto_limpo.encode('latin-1', 'replace').decode('latin-1')
    linhas = texto_limpo.split('\n')

    for linha in linhas:
        linha = linha.strip()
        
        # Pular linhas vazias ou que contenham apenas formatação de tabela/bloco
        if not linha or linha in ['---', '```']:
            pdf.ln(2)
            continue

        # --- FILTRO DE TABELAS (Transformando em frases limpas) ---
        if "|" in linha:
            if any(term in linha.lower() for term in ["area", "count", "|:---", "|---"]):
                continue
            
            colunas = [c.strip() for c in linha.split('|') if c.strip()]
            if len(colunas) >= 2:
                pdf.set_font("helvetica", '', 11)
                texto_cidade = f"- {colunas[0]}: {colunas[1]} ataques."
                # w=0 e ln=1 é a combinação mais segura contra o erro de espaço
                pdf.cell(0, 7, texto_cidade, ln=1)
            continue

        # --- IMAGENS ---
        if "![" in linha:
            match = re.search(r'\((.*?)\)', linha)
            if match:
                pdf.ln(4)
                try:
                    # x=35 centraliza, w=140 mantém margem de segurança de 30mm de cada lado
                    pdf.image(match.group(1).strip(), x=35, w=140)
                    pdf.ln(10)
                except: pass
            continue

        # --- TÍTULOS (# e ##) ---
        if linha.startswith('#'):
            pdf.ln(4)
            tamanho = 14 if linha.startswith('##') else 16
            pdf.set_font("helvetica", 'B', tamanho)
            pdf.set_text_color(43, 123, 186)
            titulo = linha.lstrip('#').strip()
            pdf.cell(0, 10, titulo, ln=1)
            pdf.set_font("helvetica", '', 11)
            pdf.set_text_color(0, 0, 0)
            continue

        # --- TEXTO NORMAL ---
        pdf.set_font("helvetica", '', 11)
        # multi_cell(0, ...) diz ao FPDF para usar toda a largura da margem esquerda até a direita
        pdf.multi_cell(0, 7, linha)
        pdf.ln(2)

    pdf.output("Relatorio_Executivo_Final_2026.pdf")
    print("Sucesso! PDF gerado sem erros de formatação.")

In [ ]:
criar_pdf_final(resultado.raw)